SETUP

In [ ]:
%pip install transformers
%pip install accelerate
%pip install safetensors
%pip install invisible-watermark>=0.2.0
%pip install omegaconf

In [ ]:
%pip install git+https://github.com/huggingface/rs.git@main

In [ ]:
%pip install 'huggingface_hub[cli]' hf_transfer


In [ ]:
%pip install -U huggingface_hub tokenizer

MODEL

In [ ]:
!wget https://civitai.com/api/download/models/148259 -O DynaVision_XL.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/129610 -O 3d_style_lora.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/127055/ -O anime_art_xl.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/94057 -O fastnegativeV2.pt

In [ ]:
!wget https://civitai.com/api/download/models/126688 -O dreamshaper.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/144566 -O samaritan3dCartoon.safetensors

SCRIPT

In [ ]:
from rs import StableDiffusionXLPipeline, UniPCMultistepScheduler,DiffusionPipeline,StableDiffusionPipeline
import torch

# base = StableDiffusionXLPipeline.from_pretrained(
#      "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
# )
base = StableDiffusionXLPipeline.from_single_file(
     "./DynaVision_XL.safetensors", 
     torch_dtype=torch.float16, 
     variant="fp16", 
     use_safetensors=True,
    scheduler="sample_dpmpp_2m_sde"
)

base.to("cuda")
# base.load_lora_weights("./3d_style_lora.safetensors")
base.load_textual_inversion("./fastnegativeV2.pt")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
refiner.to("cuda")

In [ ]:
n_steps = 35
high_noise_frac = 0.8

In [ ]:
prompt = f'''
high quality amazing detailed art of 
an 3D stylized cute little werewolf ,
full body reference image for sculting
'''
neg_prompt =f''' 
FastNegativeV2,low res, low quality, bad detail, asymmetry,text
'''
image = base(
    prompt=prompt,
  num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    # output_type="latent",
    negative_prompt=neg_prompt
).images[0]
image.show()
# image = base(
#     prompt=prompt,
#     num_inference_steps=n_steps,
#     denoising_end=high_noise_frac,
#     output_type="latent",
#     negative_prompt=neg_prompt
# ).images

# image = refiner(
#     prompt=prompt,
#     num_inference_steps=n_steps,
#     denoising_start=high_noise_frac,
#     image=image,
#     negative_prompt=neg_prompt
# ).images[0]
# image.show() 

In [ ]:
name = "werewolf"
image.save(f"/notebooks/threestudio/load/images/{name}.png")

CONTROL NET

In [ ]:
from rs import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL
from rs.utils import load_image
import numpy as np
import torch

import cv2
from PIL import Image

controlnet = ControlNetModel.from_pretrained(
    "rs/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16
)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, vae=vae, torch_dtype=torch.float16
)
pipe.enable_model_cpu_offload()



In [ ]:
prompt = "an dragon"
negative_prompt = "low quality, bad quality, sketches"



# initialize the models and pipeline
controlnet_conditioning_scale = 0.1  # recommended for good generalization
# download an image
image = Image.open("/notebooks/front.png")
# get canny image
image = np.array(image)
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)
canny_image.show()
# generate image
image = pipe(
    prompt, controlnet_conditioning_scale=controlnet_conditioning_scale, image=canny_image
).images[0]
image.show()

DEPTH

In [ ]:
import torch
import numpy as np
from PIL import Image

from transformers import DPTFeatureExtractor, DPTForDepthEstimation
from rs import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from rs.utils import load_image


depth_estimator = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas").to("cuda")
feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-hybrid-midas")
controlnet = ControlNetModel.from_pretrained(
    "rs/controlnet-depth-sdxl-1.0",
    variant="fp16",
    use_safetensors=True,
    torch_dtype=torch.float16,
).to("cuda")
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16).to("cuda")
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "Linaqruf/animagine-xl",
    controlnet=controlnet,
    vae=vae,
    variant="fp16",
    use_safetensors=True,
    torch_dtype=torch.float16,
).to("cuda")
# pipe.enable_model_cpu_offload()

def get_depth_map(image):
    image = feature_extractor(images=image, return_tensors="pt").pixel_values.to("cuda")
    with torch.no_grad(), torch.autocast("cuda"):
        depth_map = depth_estimator(image).predicted_depth

    depth_map = torch.nn.functional.interpolate(
        depth_map.unsqueeze(1),
        size=(1024, 1024),
        mode="bicubic",
        align_corners=False,
    )
    depth_min = torch.amin(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_max = torch.amax(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_map = (depth_map - depth_min) / (depth_max - depth_min)
    image = torch.cat([depth_map] * 3, dim=1)

    image = image.permute(0, 2, 3, 1).cpu().numpy()[0]
    image = Image.fromarray((image * 255.0).clip(0, 255).astype(np.uint8))
    return image




In [ ]:
import torch
import numpy as np
from PIL import Image

from transformers import DPTFeatureExtractor, DPTForDepthEstimation
from rs import ControlNetModel, StableDiffusionXLControlNetPipeline, DiffusionPipeline
from rs.utils import load_image
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=pipe.text_encoder_2,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
""" refiner.load_lora_weights("./3d_style_lora.safetensors") """
refiner.to("cuda")


In [ ]:
n_steps = 50
high_noise_frac = 0.9

In [ ]:

prompt = f'''
an cat like pokemon,masterpiece, best quality,in forest
'''
neg_prompt =f''' 
lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry
'''
image = Image.open("/notebooks/front.png").convert("RGB")
controlnet_conditioning_scale = 0.6  # recommended for good generalization

depth_image = get_depth_map(image)

image = pipe(
    prompt,
    negative_prompt=neg_prompt, 
    image=depth_image,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    output_type="latent",
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    
).images


In [ ]:
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
#    denoising_start=high_noise_frac,
    image=image,
    negative_prompt=neg_prompt
).images[0]
image.show() 

In [ ]:
%cd /notebooks/ComfyUI
!wget https://civitai.com/api/download/models/153698 -O ./models/loras/boat.safetensors

In [ ]:
%cd /notebooks/ComfyUI
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/

In [ ]:
%cd /notebooks/ComfyUI
!wget -c https://civitai.com/api/download/models/159385  -O ./models/embeddings/fastnegative.safetensors

In [ ]:
%cd /notebooks/ComfyUI
!wget -c  !wget -c https://civitai.com/api/download/models/143197 -O ./models/loras/360deg.safetensors

In [ ]:

%cd /notebooks/ComfyUI
https://civitai.com/api/download/models/143197


In [ ]:
%cd /notebooks/ComfyUI/models/embeddings/

In [ ]:
%cd /notebooks/ComfyUI
!wget https://civitai.com/api/download/models/133833 -O ./models/loras/icon.safetensors

In [ ]:
%cd /notebooks/ComfyUI
!wget https://civitai.com/api/download/models/238745 -O ./models/checkpoints/stableVideoDiffusion_v10.safetensors

In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=1
!huggingface-cli download stabilityai/stable-video-diffusion-img2vid stable-video-diffusion-img2vid.safetensors --cache-dir=/notebooks/.cache --local-dir /notebooks/ComfyUI/models/checkpoints--local-dir-use-symlinks False


In [ ]:
%cd /notebooks/ComfyUI
!wget  https://civitai.com/api/download/models/165974  -O ./models/checkpoints/blue_pencil-XL.safetensors 

In [ ]:
%cd /notebooks/ComfyUI 
!wget https://civitai.com/api/download/models/155414 -O ./models/controlnet/controlnetxlCNXL_tencentarcLineart.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!wget https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -O ./models/controlnet/control_v1p_sd15_qrcode_monster.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!wget  https://civitai.com/api/download/models/130072 -O  ./models/checkpoints/realisticVisionV51_v51VAE.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!wget https://civitai.com/api/download/models/153971 -O ./models/controlnet/controlnetxlCNXL_sargeztSoftedge.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!wget https://civitai.com/api/download/models/166909 -O  ./models/checkpoints/Juggernaut_XL.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!wget https://civitai.com/api/download/models/143906 -O  ./models/checkpoints/epiCRealism.safetensors



In [ ]:
%cd /notebooks/ComfyUI 
!wget https://civitai.com/api/download/models/126688 -O  ./models/checkpoints/dreamshaper.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!rm ./models/controlnet/controlnetxlCNXL_kohyaBlur.safetensors
!wget https://civitai.com/api/download/models/158411 -O ./models/controlnet/controlnetxlCNXL_kohyaBlur.safetensors

In [ ]:
%cd /notebooks/ComfyUI 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://civitai.com/api/download/models/46846 -d ./models/checkpoints/ -o "ReV Animated_sd15.safetensors"


In [ ]:
%cd /notebooks/ComfyUI 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://civitai.com/api/download/models/101966 -d ./models/checkpoints/ -o "CarDos Animated_sd15.safetensors"

In [ ]:
%cd /notebooks/ComfyUI 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/16869?type=Model&format=SafeTensor&size=full&fp=fp16" -d ./models/loras/ -o "Sung Jinwoo (Solo Levelling) LoRA.safetensors"

In [ ]:
%cd /notebooks/ComfyUI 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/39430" -d ./models/loras/ -o "Solo leveling shadows_sd15.safetensors"


In [ ]:
%cd /notebooks/ComfyUI 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/128713?type=Model&format=SafeTensor&size=pruned&fp=fp16" -d ./models/checkpoints/ -o "DreamShaper34K_sd15.safetensors"


In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hotshotco/Hotshot-XL/resolve/main/hsxl_temporal_layers.safetensors -d  /notebooks/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models -o hsxl_temporal_layers.safetensors 

In [ ]:
!rm /notebooks/ComfyUI/temp/ -rf

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/162380" -d  /notebooks/ComfyUI/models/checkpoints/ -o "Mysterious | SDXL.safetensors" 

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  "https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/resolve/main/unet/diffusion_pytorch_model.safetensors" -d  /notebooks/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models -o "text-to-video-ms-1.7b.safetensors" 

In [ ]:
%cd /notebooks/ComfyUI 
!wget  https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_opticalpattern.safetensors -O ./models/controlnet/control_v01u_sdxl_opticalpattern.safetensors

DOWNLOAD MODEL CIVITAI

In [ ]:
link="https://civitai.com/api/download/models/234202"
name="stableVideoDiffusion_img2vidXt.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/models/checkpoints/ -o "{name}"

DOWNLOAD LORA

In [ ]:
link="https://civitai.com/api/download/models/198530"
name=""
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/models/loras -o "{name}"

DOWNLOAD CONTROL NET

In [ ]:
link="https://huggingface.co/CiaraRowles/controlnet-temporalnet-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors"
name="diffusion_pytorch_model.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/models/controlnet -o "{name}"

DOWNLOAD EMBBEDING

In [ ]:
link="https://civitai.com/api/download/models/175819"
name="unaestheticXL_Sky3.1.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/models/embeddings -o "{name}"

In [ ]:
%cd /notebooks/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models 
!rm diffusion_pytorch_model.safetensors 
!ls

In [ ]:
# Define the download links, their target directories, and their filenames
download_links = {
    # For SD1.5
    "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter_sd15.bin"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter_sd15_light.bin"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter-plus_sd15.bin"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter-plus-face_sd15.bin"),
    
    # For SDXL
    "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter_sdxl.bin"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter_sdxl_vit-h.bin"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.bin": ("/notebooks/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models", "ip-adapter-plus_sdxl_vit-h.bin"),
    
    # Image encoders
    "https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors": ("/notebooks/ComfyUI/models/clip_vision/", "model_sd1.5.safetensors"),
    "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors": ("/notebooks/ComfyUI/models/clip_vision/", "model_sdxl.safetensors")
}

# Download the files
for link, (path, filename) in download_links.items():
    !aria2c -d {path} -o {filename} {link}

DOWNLOAD ANIMDIFF MODEL

In [ ]:
link="https://huggingface.co/hotshotco/Hotshot-XL/blob/main/hsxl_temporal_layers.f16.safetensors"
name="hsxl_temporal_layers.f16.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models  -o "{name}"

DOWNLOAD ANIMDIFF LORA

In [ ]:
link="https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt"
name="v2_lora_ZoomOut.ckpt"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{link}" -d  /notebooks/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o "{name}"

In [ ]:
%cd /notebooks/gaussian-splatting/output
!rm -rf *